In [1]:
import os
import pandas as pd
import numpy as np
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
import timeit

### Train Data

In [2]:
maindir = "data/" # Directory with your files
traincsv = maindir+"/Train.csv"
testcsv = maindir+"/Test.csv"

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
train = pd.read_csv(traincsv)
train.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,absorbance40,absorbance41,absorbance42,absorbance43,absorbance44,absorbance45,absorbance46,absorbance47,absorbance48,absorbance49,absorbance50,absorbance51,absorbance52,absorbance53,absorbance54,absorbance55,absorbance56,absorbance57,absorbance58,absorbance59,absorbance60,absorbance61,absorbance62,absorbance63,absorbance64,absorbance65,absorbance66,absorbance67,absorbance68,absorbance69,absorbance70,absorbance71,absorbance72,absorbance73,absorbance74,absorbance75,absorbance76,absorbance77,absorbance78,absorbance79,absorbance80,absorbance81,absorbance82,absorbance83,absorbance84,absorbance85,absorbance86,absorbance87,absorbance88,absorbance89,absorbance90,absorbance91,absorbance92,absorbance93,absorbance94,absorbance95,absorbance96,absorbance97,absorbance98,absorbance99,absorbance100,absorbance101,absorbance102,absorbance103,absorbance104,absorbance105,absorbance106,absorbance107,absorbance108,absorbance109,absorbance110,absorbance111,absorbance112,absorbance113,absorbance114,absorbance115,absorbance116,absorbance117,absorbance118,absorbance119,absorbance120,absorbance121,absorbance122,absorbance123,absorbance124,absorbance125,absorbance126,absorbance127,absorbance128,absorbance129,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,donation_id
0,0.520883,0.528200,0.529795,0.537382,0.540265,0.546260,0.550882,0.558865,0.571352,0.585775,0.600688,0.612949,0.617855,0.621710,0.622049,0.623373,0.620610,0.619479,0.616463,0.612414,0.609885,0.605408,0.602115,0.597710,0.593485,0.592332,0.588680,0.587396,0.585577,0.585244,0.583597,0.584460,0.584548,0.586147,0.589067,0.591360,0.595636,0.599054,0.602856,0.607817,0.614662,0.621709,0.630737,0.645748,0.665720,0.699577,0.729288,0.758677,0.778546,0.793011,0.802464,0.806903,0.812957,0.817343,0.820385,0.822908,0.822506,0.823586,0.821152,0.818050,0.816039,0.812601,0.809218,0.806484,0.802914,0.800798,0.798346,0.795475,0.794360,0.794957,0.795533,0.798059,0.800934,0.804687,0.809206,0.814001,0.824323,0.832220,0.842657,0.855244,0.868704,0.889791,0.910066,0.931412,0.954060,0.976112,0.998979,1.028305,1.050609,1.070898,1.087282,1.110074,1.141318,1.173962,1.215623,1.270062,1.336545,1.424083,1.482955,1.543775,1.584717,1.615581,1.639260,1.667806,1.671933,1.693926,1.685613,1.695639,1.703657,1.706169,1.708177,1.708638,1.700817,1.698755,1.697645,1.698235,1.686467,1.682201,1.677926,1.664489,1.662983,1.656813,1.642451,1.635306,1.622732,1.609448,1.599344,1.594260,1.569761,1.561136,1.554982,1.544208,1.532990,1.507301,1.501713,1.499864,1.482482,1.478964,1.468505,1.454910,1.434286,1.443249,1.417403,1.418581,1.403326,1.395943,1.386288,1.381532,1.372256,1.367083,1.362999,1.353013,1.345374,1.341283,1.323804,1.331071,1.332295,1.328928,1.309800,1.328064,1.329464,1.314664,1.313583,1.313329,1.312243,1.310330,1.353629,1.336609,1.319344,1.278294,0,0.0026,39.27,44.17,low,ok,high,2250
1,0.529109,0.534852,0.541075,0.544269,0.546594,0.5551

In [5]:
train.columns

Index(['absorbance0', 'absorbance1', 'absorbance2', 'absorbance3',
       'absorbance4', 'absorbance5', 'absorbance6', 'absorbance7',
       'absorbance8', 'absorbance9',
       ...
       'absorbance168', 'absorbance169', 'id', 'std', 'temperature',
       'humidity', 'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human', 'donation_id'],
      dtype='object', length=178)

In [6]:
dependent_var = train[['hdl_cholesterol_human', 'hemoglobin(hgb)_human','cholesterol_ldl_human']]
dependent_var.head()

,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,low,ok,high
1,low,ok,high
2,low,ok,high
3,low,ok,high
4,low,ok,high


In [7]:
X_train = train.drop(['hdl_cholesterol_human', 'hemoglobin(hgb)_human','cholesterol_ldl_human','id','donation_id'],axis=1)

In [8]:
X_train.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,absorbance40,absorbance41,absorbance42,absorbance43,absorbance44,absorbance45,absorbance46,absorbance47,absorbance48,absorbance49,absorbance50,absorbance51,absorbance52,absorbance53,absorbance54,absorbance55,absorbance56,absorbance57,absorbance58,absorbance59,absorbance60,absorbance61,absorbance62,absorbance63,absorbance64,absorbance65,absorbance66,absorbance67,absorbance68,absorbance69,absorbance70,absorbance71,absorbance72,absorbance73,absorbance74,absorbance75,absorbance76,absorbance77,absorbance78,absorbance79,absorbance80,absorbance81,absorbance82,absorbance83,absorbance84,absorbance85,absorbance86,absorbance87,absorbance88,absorbance89,absorbance90,absorbance91,absorbance92,absorbance93,absorbance94,absorbance95,absorbance96,absorbance97,absorbance98,absorbance99,absorbance100,absorbance101,absorbance102,absorbance103,absorbance104,absorbance105,absorbance106,absorbance107,absorbance108,absorbance109,absorbance110,absorbance111,absorbance112,absorbance113,absorbance114,absorbance115,absorbance116,absorbance117,absorbance118,absorbance119,absorbance120,absorbance121,absorbance122,absorbance123,absorbance124,absorbance125,absorbance126,absorbance127,absorbance128,absorbance129,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,std,temperature,humidity
0,0.520883,0.528200,0.529795,0.537382,0.540265,0.546260,0.550882,0.558865,0.571352,0.585775,0.600688,0.612949,0.617855,0.621710,0.622049,0.623373,0.620610,0.619479,0.616463,0.612414,0.609885,0.605408,0.602115,0.597710,0.593485,0.592332,0.588680,0.587396,0.585577,0.585244,0.583597,0.584460,0.584548,0.586147,0.589067,0.591360,0.595636,0.599054,0.602856,0.607817,0.614662,0.621709,0.630737,0.645748,0.665720,0.699577,0.729288,0.758677,0.778546,0.793011,0.802464,0.806903,0.812957,0.817343,0.820385,0.822908,0.822506,0.823586,0.821152,0.818050,0.816039,0.812601,0.809218,0.806484,0.802914,0.800798,0.798346,0.795475,0.794360,0.794957,0.795533,0.798059,0.800934,0.804687,0.809206,0.814001,0.824323,0.832220,0.842657,0.855244,0.868704,0.889791,0.910066,0.931412,0.954060,0.976112,0.998979,1.028305,1.050609,1.070898,1.087282,1.110074,1.141318,1.173962,1.215623,1.270062,1.336545,1.424083,1.482955,1.543775,1.584717,1.615581,1.639260,1.667806,1.671933,1.693926,1.685613,1.695639,1.703657,1.706169,1.708177,1.708638,1.700817,1.698755,1.697645,1.698235,1.686467,1.682201,1.677926,1.664489,1.662983,1.656813,1.642451,1.635306,1.622732,1.609448,1.599344,1.594260,1.569761,1.561136,1.554982,1.544208,1.532990,1.507301,1.501713,1.499864,1.482482,1.478964,1.468505,1.454910,1.434286,1.443249,1.417403,1.418581,1.403326,1.395943,1.386288,1.381532,1.372256,1.367083,1.362999,1.353013,1.345374,1.341283,1.323804,1.331071,1.332295,1.328928,1.309800,1.328064,1.329464,1.314664,1.313583,1.313329,1.312243,1.310330,1.353629,1.336609,1.319344,1.278294,0.0026,39.27,44.17
1,0.529109,0.534852,0.541075,0.544269,0.546594,0.555183,0.561367,0.567637,0.582427,0.595801,0.608619,0.619217,0.625162,0.627111,0.629326,0.628116,0.62781

In [9]:
y_var = dependent_var.replace({"low":0,"ok":1,"high":2})

In [10]:
y_var.head()

,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,0,1,2
1,0,1,2
2,0,1,2
3,0,1,2
4,0,1,2


In [11]:
y_hdl = y_var['hdl_cholesterol_human']
y_hem = y_var['hemoglobin(hgb)_human']
y_cho = y_var['cholesterol_ldl_human']

### Test data

In [12]:
test = pd.read_csv(testcsv)
test.head()

,Reading_ID,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,absorbance40,absorbance41,absorbance42,absorbance43,absorbance44,absorbance45,absorbance46,absorbance47,absorbance48,absorbance49,absorbance50,absorbance51,absorbance52,absorbance53,absorbance54,absorbance55,absorbance56,absorbance57,absorbance58,absorbance59,absorbance60,absorbance61,absorbance62,absorbance63,absorbance64,absorbance65,absorbance66,absorbance67,absorbance68,absorbance69,absorbance70,absorbance71,absorbance72,absorbance73,absorbance74,absorbance75,absorbance76,absorbance77,absorbance78,absorbance79,absorbance80,absorbance81,absorbance82,absorbance83,absorbance84,absorbance85,absorbance86,absorbance87,absorbance88,absorbance89,absorbance90,absorbance91,absorbance92,absorbance93,absorbance94,absorbance95,absorbance96,absorbance97,absorbance98,absorbance99,absorbance100,absorbance101,absorbance102,absorbance103,absorbance104,absorbance105,absorbance106,absorbance107,absorbance108,absorbance109,absorbance110,absorbance111,absorbance112,absorbance113,absorbance114,absorbance115,absorbance116,absorbance117,absorbance118,absorbance119,absorbance120,absorbance121,absorbance122,absorbance123,absorbance124,absorbance125,absorbance126,absorbance127,absorbance128,absorbance129,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity
0,ID_KWUAQLRM,ID_2982,0.517951,0.520508,0.526852,0.531611,0.536816,0.543828,0.547761,0.554379,0.565622,0.575762,0.590253,0.600692,0.605104,0.608611,0.608169,0.606985,0.606590,0.603814,0.600409,0.597802,0.593389,0.589956,0.585429,0.583659,0.578307,0.574303,0.571248,0.570985,0.568520,0.566437,0.564261,0.563907,0.563427,0.564336,0.565472,0.567827,0.570957,0.572206,0.576374,0.578900,0.584714,0.590129,0.598283,0.610942,0.629885,0.663671,0.691244,0.717517,0.738406,0.750543,0.760688,0.765623,0.768486,0.773027,0.775859,0.780021,0.779471,0.779183,0.778051,0.776737,0.772796,0.767294,0.761112,0.757747,0.753633,0.750178,0.747777,0.743685,0.743375,0.741121,0.741824,0.744002,0.747120,0.750457,0.755227,0.760167,0.768432,0.777124,0.786463,0.797902,0.812258,0.833389,0.850751,0.871127,0.894107,0.916401,0.938588,0.967226,0.987011,1.008324,1.027563,1.050746,1.082240,1.118286,1.161036,1.215094,1.277405,1.359670,1.420495,1.467652,1.505196,1.530854,1.558123,1.576174,1.583064,1.592206,1.588731,1.601169,1.605713,1.603868,1.599217,1.603527,1.599267,1.605364,1.602302,1.596339,1.588176,1.581886,1.577644,1.572630,1.561670,1.555143,1.550746,1.537073,1.530301,1.520875,1.506511,1.497650,1.481027,1.479365,1.465003,1.449586,1.441030,1.427545,1.416794,1.402706,1.391447,1.385020,1.373256,1.362781,1.345200,1.341810,1.333717,1.328438,1.313325,1.305895,1.293996,1.292892,1.285608,1.275789,1.267682,1.263906,1.259461,1.249615,1.244618,1.242537,1.238426,1.233983,1.235212,1.234433,1.234971,1.223950,1.235415,1.234605,1.226695,1.210430,1.214422,1.200332,1.198297,1.248182,0,0.0027,37.09,27.22
1,ID_FN8SIVJJ,ID_2982,0.517839,0.522367,0.525186,0.534661,0.541900,0.546180,0.551687,0.556753,0.566446,0.

In [13]:
# save it ids before dropping them
idx = test.donation_id.values
X_test = test.drop(["Reading_ID","id","donation_id"], axis=1)

In [15]:
X_test.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,absorbance40,absorbance41,absorbance42,absorbance43,absorbance44,absorbance45,absorbance46,absorbance47,absorbance48,absorbance49,absorbance50,absorbance51,absorbance52,absorbance53,absorbance54,absorbance55,absorbance56,absorbance57,absorbance58,absorbance59,absorbance60,absorbance61,absorbance62,absorbance63,absorbance64,absorbance65,absorbance66,absorbance67,absorbance68,absorbance69,absorbance70,absorbance71,absorbance72,absorbance73,absorbance74,absorbance75,absorbance76,absorbance77,absorbance78,absorbance79,absorbance80,absorbance81,absorbance82,absorbance83,absorbance84,absorbance85,absorbance86,absorbance87,absorbance88,absorbance89,absorbance90,absorbance91,absorbance92,absorbance93,absorbance94,absorbance95,absorbance96,absorbance97,absorbance98,absorbance99,absorbance100,absorbance101,absorbance102,absorbance103,absorbance104,absorbance105,absorbance106,absorbance107,absorbance108,absorbance109,absorbance110,absorbance111,absorbance112,absorbance113,absorbance114,absorbance115,absorbance116,absorbance117,absorbance118,absorbance119,absorbance120,absorbance121,absorbance122,absorbance123,absorbance124,absorbance125,absorbance126,absorbance127,absorbance128,absorbance129,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,std,temperature,humidity
0,0.517951,0.520508,0.526852,0.531611,0.536816,0.543828,0.547761,0.554379,0.565622,0.575762,0.590253,0.600692,0.605104,0.608611,0.608169,0.606985,0.606590,0.603814,0.600409,0.597802,0.593389,0.589956,0.585429,0.583659,0.578307,0.574303,0.571248,0.570985,0.568520,0.566437,0.564261,0.563907,0.563427,0.564336,0.565472,0.567827,0.570957,0.572206,0.576374,0.578900,0.584714,0.590129,0.598283,0.610942,0.629885,0.663671,0.691244,0.717517,0.738406,0.750543,0.760688,0.765623,0.768486,0.773027,0.775859,0.780021,0.779471,0.779183,0.778051,0.776737,0.772796,0.767294,0.761112,0.757747,0.753633,0.750178,0.747777,0.743685,0.743375,0.741121,0.741824,0.744002,0.747120,0.750457,0.755227,0.760167,0.768432,0.777124,0.786463,0.797902,0.812258,0.833389,0.850751,0.871127,0.894107,0.916401,0.938588,0.967226,0.987011,1.008324,1.027563,1.050746,1.082240,1.118286,1.161036,1.215094,1.277405,1.359670,1.420495,1.467652,1.505196,1.530854,1.558123,1.576174,1.583064,1.592206,1.588731,1.601169,1.605713,1.603868,1.599217,1.603527,1.599267,1.605364,1.602302,1.596339,1.588176,1.581886,1.577644,1.572630,1.561670,1.555143,1.550746,1.537073,1.530301,1.520875,1.506511,1.497650,1.481027,1.479365,1.465003,1.449586,1.441030,1.427545,1.416794,1.402706,1.391447,1.385020,1.373256,1.362781,1.345200,1.341810,1.333717,1.328438,1.313325,1.305895,1.293996,1.292892,1.285608,1.275789,1.267682,1.263906,1.259461,1.249615,1.244618,1.242537,1.238426,1.233983,1.235212,1.234433,1.234971,1.223950,1.235415,1.234605,1.226695,1.210430,1.214422,1.200332,1.198297,1.248182,0.0027,37.09,27.22
1,0.517839,0.522367,0.525186,0.534661,0.541900,0.546180,0.551687,0.556753,0.566446,0.578208,0.591039,0.601620,0.605895,0.609568,0.610188,0.608635,0.60771

In [14]:
X_test.shape, X_train.shape

((3660, 173), (29160, 173))

### Train Model

* Lightgbm

In [ ]:
#### y_hdl = y_var['hdl_cholesterol_human']

In [19]:
# evaluate the Lightgbm model
from lightgbm import LGBMClassifier
start = timeit.default_timer()
cv = RepeatedKFold(n_splits=10, random_state=1)
model_lgb_hdl = LGBMClassifier()
model_lgb_hdl = model_lgb_hdl.fit(X_train, y_hdl)
scores = cross_val_score(model_lgb_hdl, X_train, y_hdl,scoring='accuracy', cv=cv)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))# fit the model on the whole dataset
stop = timeit.default_timer()
print(stop - start)

1.00 accuracy with a standard deviation of 0.00
815.1163365000001


In [ ]:
### y_hem = y_var['hemoglobin(hgb)_human']

In [ ]:
# evaluate the Lightgbm model
from lightgbm import LGBMClassifier
start = timeit.default_timer()
cv = RepeatedKFold(n_splits=10, random_state=1)
model_lgb_hem = LGBMClassifier()
model_lgb_hem = model_lgb_hem.fit(X_train, y_hem)
scores = cross_val_score(model_lgb_hem, X_train, y_hem,scoring='accuracy', cv=cv)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))# fit the model on the whole dataset
stop = timeit.default_timer()
print(stop - start)

In [ ]:
#### y_cho = y_var['cholesterol_ldl_human']

In [ ]:
# evaluate the Lightgbm model
from lightgbm import LGBMClassifier
start = timeit.default_timer()
cv = RepeatedKFold(n_splits=10, random_state=1)
model_lgb_cho = LGBMClassifier()
model_lgb_cho = model_lgb_cho.fit(X_train, y_cho)
scores = cross_val_score(model_lgb_cho, X_train, y_cho,scoring='accuracy', cv=cv)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))# fit the model on the whole dataset
stop = timeit.default_timer()
print(stop - start)